<a href="https://colab.research.google.com/github/davidnoone/ENVPHYS300/blob/main/ENVPHYS300_2025_Lab3_E%2BM_cycles_Part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ENVPHYS 300 Lab3: Energy and Momentum cycles


## The atmospheric general circulation: Part 2


The general circulation of the atmosphere is the description of the average patterns which links wind and temperature distributions and provide energy transport from the tropics to higher latitudes. It captures the way in which the atmosphere responses to the energy imbalance while also conserving mass and momentum.
Here we develop an analysis of the energy and momentum budgets and assess the role of atmospheric waves in the general circulation.


We will use data from the NCEP/DoE Reanalysis project to analyze the atmospheric transports. The archive of data can be found here.
https://psl.noaa.gov/data/gridded/data.ncep.reanalysis2.html


The python tasks contained here are associated with taking different types of averages, performing a decomposition of the atmospheric flow, and associating the resulting diagnostics with the global energy and momentum balance.

In [ ]:
# initalize te environment
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt

import datetime

try:
    import netCDF4 as ncdf
except:
    print('NetCDF needs to be installed: this will take a few moments')
    !pip install netCDF4
    import netCDF4 as ncdf

import sys
import time as time_lib
import urllib.request

import xarray as xr

# Some constants
gravity = 9.81          # acceleration due to gravity,  m/s2
cpair = 1005            # specific heat capacity of dry air, J/K/kg
latvap = 2.26e6         # latent heat of vaporization, J/kg
epsilon = 0.622         # ratio of molecular weights for water vapor
Tzero = 273.16          # temperature K of 0C.


def reporthook(count, block_size, total_size):
    global start_time
    if count == 0:
        start_time = time_lib.time()
        return
    duration = time_lib.time() - start_time
    progress_size = int(count * block_size)
    speed = int(progress_size / (1024 * duration))
    percent = int(count * block_size * 100 / total_size)
    sys.stdout.write("\r...%d%%, %d MB, %d KB/s, %d seconds passed" %
                    (percent, progress_size / (1024 * 1024), speed, duration))
    sys.stdout.flush()

##PART 2: Zonal energy and momentum balance (separate python sheet for next week)


Obtain data for the year 2024 for v, and T . We want to use data at 6 hourly intervals. This will be approximately 1 GB of data! It will take a while do download (take care, no need to rerun those python cells!)

Our goals is to determine the role of atmospheric motions in the movement of energy and momentum. We will consider the poleward transport of energy and momentum by the zonal mean flow and by eddies.

The poleward flux can be written as $vT$ for temperature and $uv$ for momentum. Notice the units.


We will deconstruct the total flow in to a zonal mean and a deviation. Consider temperature as:

$$
T = \bar{T} + T'
$$

with the bar denoting the zonal mean, and the prime the deviation from that.


Therefore the poleward flux of temperature can be decompose as $\overline{vT} = \bar{v} \bar{T}  + \overline{v'T'}$. The first term is named the zonal mean transport, and the second is the eddy transport. (Note that one could similarly decompose the water vapor mixing ratio, $q$, and the eastward momntum, $u$, or other quantities in this way.)





TASK: Using data from 2024, compute the zonal means at each pressure level, and generate the deviations u', v' and T' (at each longitude), then compute the zonal mean of the eddy and mean transport of heat, moisture and zonal momentum.


TASK: For each season (DJF and JJA) compute the zonal mean of the meridional (northward) flux of easterly momentum and temperature.
This should give you 6 figures for each season.

$\overline{vT}, \bar{v} \bar{T},  \overline{v'T'}$

$\overline{uv}, \bar{u} \bar{v},  \overline{u'v'}$


[Hint: Start with just temperature! Once you are happy you can eaily redo it with u and q]

[Hint 2: I recommend setting up a plot function that allows you to make a 6 pannel figure: DJF on the left, JJA on the right. Then rows of total, zonal mean and eddy fluxes.]

On your figures, be sure to state the units.

In [ ]:
# Handy little "read" function
def read_netcdf_variable(filename,varname):
    print('Reading ['+varname+'] from',filename)
    with ncdf.Dataset(filename,'r') as ncfile:    # open the file and read the data...
        lons  = ncfile.variables['lon'][:]            # read the longitudes
        lats  = ncfile.variables['lat'][:]            # read the latitudes
        levs  = ncfile.variables['level'][:]            # read the pressure levels
        time  = ncfile.variables['time'][:]           # read the time axis

        data  = ncfile.variables[varname][:,:,:]        # read the 500 hPa height

    return data, lons,lats,levs,time

In [ ]:
# Example: get the data

#Temperature
URL_air = "https://downloads.psl.noaa.gov/Datasets/ncep.reanalysis2/pressure/air.2024.nc"
filename_air = 'air_6hourly.nc'

# Wind
URL_vwnd = "https://downloads.psl.noaa.gov/Datasets/ncep.reanalysis2/pressure/vwnd.2024.nc"
filename_vwnd = 'vwnd_6hourly.nc'

# THIS WILL TAKE A WHILE TO DOWN LOAD - run this ONCE
done_download = False
if (not done_download):
    #urllib.request.urlretrieve(URL_air,filename_air,reporthook)
    #urllib.request.urlretrieve(URL_vwnd,filename_vwnd,reporthook)

    done_download = True
else:
    print('Files already downloaded')


In [ ]:
#vwnd, lons,lats,levs,time = read_netcdf_variable(filename_vwnd,'vwnd')
temp, lons,lats,levs,time = read_netcdf_variable(filename_air,'air')


nlon = len(lons); nlat = len(lats); nlev = len(levs); ntime = len(time)
print('NLON:',nlon,' NLAT:',nlat, 'NLEV:',nlev,' NTIME:',ntime)

# mixing ratio conversion

print('Success getting all the data arrays thath you need.')

In [ ]:
#
# Sample code to select the seasons
#
temp_JJA = temp[152*4:244*4-1,:,:,:]          # June - August
temp_DJF = np.concatenate((temp[0:60*4-1, :, :, :], temp[335*4:365*4-1, :, :, :]), axis=0)

print('Seasonal arrays:',np.shape(temp_JJA),np.shape(temp_DJF),)


In [ ]:
#
# Some helper functions
#
def calculate_zonal_mean(data):
    """ Function to compute the zonal mean, time mean: of data[ntime,nlev,nlat,nlon] """
    time_mean = np.mean(data, axis=0)
    zonal_mean = np.mean(time_mean, axis=-1)
    return zonal_mean

def remove_zonal_mean(data,zonal_mean):
    """Removes zonal mean to compute anomalies: e.g., T' = T - T_mean """
    data -= zonal_mean[np.newaxis, :, :, np.newaxis]
    return data         # Notice this is done "in place" (overwrites original data)

def get_pdel(levels_hPa):
    """ Computes pressure thickness (in Pa) based on levels, in hPa"""
    half_levels = (levels_hPa[:-1] + levels_hPa[1:]) / 2

    plo = 1.5*levels_hPa[0] - 0.5*levels_hPa[1]
    half_levels = np.insert(half_levels, 0, plo)                # "ground"
    half_levels = np.append(half_levels, 0.5*levels_hPa[-1])   # TOA
    pdel = (half_levels[:-1]-half_levels[1:])*100.            # Convert hPa-> Pa
    #print("Pdel",pdel)      # check!
    return pdel

def calculate_vertical_integral(zonal_data,pdel):
    """ Computes the vertical integral sum(zonal_data dp_k)/gravity"""
    gravity = 9.81
    (nz,ny) = np.shape(zonal_data)
    vertical_integral = np.zeros(ny)
    for k in range(nz):
        vertical_integral[:] = vertical_integral[:] + np.squeeze(zonal_data[k,:]) * pdel[k]
    vertical_integral = vertical_integral/gravity
    return vertical_integral




In [ ]:
#
# Examples just for temperature
#
T_zonal = calculate_zonal_mean(temp)
T_prime = remove_zonal_mean(temp, T_zonal)

print('Dimensions for zonal mean T: ',T_zonal.shape)
print('Dimensions for T deviations:',T_prime.shape)

#
# Compute zonal mean heat flux and zonal mean of eddy heat flux
#
#vT_zonal = v_zonal * T_zonal
#vT_eddy = calculate_zonal_mean(v_prime * T_prime)         # e.g., zonal mean of (v'T')
#vT_total = vT_zonal + vT_eddy
#print('Dimensions for zonal mean heat flux: vT_zonal,',vT_zonal.shape)
#print('Dimensions for eddy heat flux are  : vT_eddy,',vT_eddy.shape)

#
# [Your code here]
#





In [ ]:
#
# Example 6 panel figure
#
def six_panel_plot(lats,levs,six_data,six_labs):
    tickvals = [1000, 700, 500, 200, 100, 50]      # tick marks in hPa
    ticklabels = [str(t) for t in tickvals]                 # Convert to string for labels

    fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(8, 11))
    axes = axes.flatten()

    for i, ax in enumerate(axes):
        one_data = six_data[i]
        ax.contourf(lats,levs,one_data)  # Example sine waves
        ax.set_yscale('log')
        ax.set_ylim(1000, 50)
        ax.set_yticks(tickvals)
        ax.set_yticklabels(ticklabels)
        ax.set_ylabel(f"Pressure (hPa)")
        ax.set_xlabel(f"Latitude (degrees)")
        ax.set_title(six_labs[i])

    plt.tight_layout()
    fig.show()
    return

# Make a list of data to plot: stack up titles and data arrays in a list
six_labs = ['DJF vT_total','JJA vt_total', \
            'DJF vT_zonal','JJA vt_zonal', \
            'DJF vT_eddy' ,'JJA vt_eddy' ]
six_data = [T_zonal, T_zonal, T_zonal, T_zonal, T_zonal, T_zonal]

# Function call to make the plot
#six_panel_plot(lats,levs,six_data,six_labs)

##Zonal mean momentum

Earlier we argued that the atmosphere must be near thermal wind balance. For a balance there must be two (at least!) equal and opposite acceleration.


TASK: Generate a plot showing the easterly acceleration due to the Coriolis force.

TASK: Generate a plot showing the easterly acceleration due to the transport of momentum by eddies (i.e. the divergence of $\overline{u'v'}$). (Hint: be cautions of the cosine needed for the divergence!)



In [ ]:
#
# Your code here
#

Questions

a) Comment on similarities and differences of these. Are there locations where they are almost equal?
(you may wish to plot a difference chart to help identify the balances)



[Your answers]

b) Comment on locations where they appear not to be balanced.




[your answer here]

c) Considering the momentum balance above, explain the sources and sinks of momentum! Where does the atmospheric momentum come from? Where does it go? What process (processes) are responsible for these sources and sinks.

[your answer here]

## Global energy transport and balance

Starting from your zonal mean cross sections, compute the vertical integral to show the total atmospheric transport of sensible heat.

It will be useful to consider the zonal, and vertical and time average of the thermodynamic equation:


$$
c_p \frac{\partial T}{\partial t} =
   - c_p \frac{\partial (\bar{v}\bar{T})}{\partial y} +
   - c_p \frac{\partial (\overline{v'T'})}{\partial y} +
   \alpha \omega + Q
$$

Where $c_p = 1005 J/K/kg $is the specific heat capacity of dry air.


TASK: Calculate the heating rates associated with the total transport of heat in the atmospheric column. This can be derived as the divergence of the flux of sensible heat.

You should produce a line plot in units of W/m2 with 3 lines for each of the sensible due to the: 1) zonal mean, 2) eddy and 3) total.






In [ ]:

## Example vertical integral
#pdel = get_pdel(levs)
#vT_zonal_int = calculate_vertical_integral(vT_zonal,pdel)
#print('vT vertical integral:',np.shape(vT_zonal_int))        # should be 1d, by latitude


#
# your code here
#


#
# You line plot code here
#


Questions:

In the above, you’ve described the cycle of both energy and momentum in the atmosphere.

a) Describe the relative role of the zonal mean transport and the eddy transport on the sensible heat transport. Describe which is larger. Are all the components of energy transports always poleward?







[Your answers]

b) Reflecting on global energy balance, explain how your result for the total energy transport would be related to the top of the atmosphere radiative balance (i.e., incoming solar minus outgoing longwave radiation).

[your answer here]

c) What, if anything, is missing in your total accounting for energy transport.

[your anser here]